<a href="https://colab.research.google.com/github/swaroopsaikuchi/DL_models/blob/main/Removing_Prefixes_from_Names_using_Sequence_to_Sequence_Encoder_Decoder_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
from sklearn.utils import shuffle
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, Dense,Embedding
from keras.models import Model,load_model
from keras.utils import plot_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
import pickle as pkl
import numpy as np

2024-07-05 10:32:57.210756: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-05 10:32:57.260461: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 10:32:57.260522: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 10:32:57.260571: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 10:32:57.272393: I tensorflow/core/platform/cpu_feature_g

In [ ]:
import tensorflow as tf
gpu_available = tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2024-07-05 10:32:59.985543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /device:GPU:0 with 78958 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0


In [ ]:
def detect_and_clean(name):
    prefixes = ["Mr.", "Ms.", "Dr.", "Srimathi", "Mrs.", "Prof.","dom."]
    for prefix in prefixes:
        if name.startswith(prefix):
            return name[len(prefix):].strip()
    return name
df = pd.read_excel('id_name_1001.xlsx')
df

,id,name
0,1,dom.swaroop
1,2,Mr. Alice
2,3,Mr. Frank
3,4,Grace
4,5,Alice
...,...,...
995,996,Ivy
996,997,Miss Frank
997,998,Prof. Ivy
998,999,Mr. Eve


In [ ]:
clean_names = []
for name in df['name']:
    clean_name = detect_and_clean(name)
    clean_names.append(clean_name)
df['cleaned_names'] = clean_names

In [ ]:
df

,id,name,cleaned_names
0,1,dom.swaroop,swaroop
1,2,Mr. Alice,Alice
2,3,Mr. Frank,Frank
3,4,Grace,Grace
4,5,Alice,Alice
...,...,...,...
995,996,Ivy,Ivy
996,997,Miss Frank,Miss Frank
997,998,Prof. Ivy,Ivy
998,999,Mr. Eve,Eve


In [ ]:
names = df['name'].to_list()
cleaned_names = df['cleaned_names'].to_list()
cleaned_names = ['<start>' + name + '<end>' for name in cleaned_names]
tokenizer = Tokenizer(char_level=True, filters='')
tokenizer.fit_on_texts(names + cleaned_names)
tokenizer.word_index['<start>'] = len(tokenizer.word_index) + 1
tokenizer.word_index['<end>'] = len(tokenizer.word_index) + 1
total_chars = len(tokenizer.word_index) + 1

def encode_sequences(tokenizer, lines, max_length):
    # integer encode sequences
    encoded = tokenizer.texts_to_sequences(lines)
    # pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

max_length_input = max([len(name) for name in names])
max_length_output = max([len(name) for name in cleaned_names])

X = encode_sequences(tokenizer, names, max_length_input)
y = encode_sequences(tokenizer, cleaned_names, max_length_output)

# One-hot encode the outputs
y = tf.keras.utils.to_categorical(y, num_classes=total_chars)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Define the model
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(total_chars, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(total_chars, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(total_chars, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

2024-07-05 10:33:00.577820: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78958 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            7424      ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            7424      ['input_2[0][0]']             
                                                                                              

In [ ]:
# Prepare decoder input data
decoder_input_data = np.zeros((X.shape[0], max_length_output))

# Shift the target sequence by one time step
decoder_input_data[:, 1:] = np.argmax(y, axis=-1)[:, :-1]
decoder_input_data[:, 0] = tokenizer.word_index['<start>']

# Train the model
model.fit([X, decoder_input_data], y, batch_size=64, epochs=100, validation_split=0.2)


Epoch 1/100


2024-07-05 10:33:05.465368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-07-05 10:33:05.638266: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0e10023520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-05 10:33:05.638339: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-07-05 10:33:05.647518: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-05 10:33:05.736173: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 [==============================] - 5s 85ms/step - loss: 2.6867 - val_loss: 2.0639
Epoch 2/100
13/13 [==============================] - 0s 13ms/step - loss: 1.7952 - val_loss: 1.5783
Epoch 3/100
13/13 [==============================] - 0s 14ms/step - loss: 1.5079 - val_loss: 1.5370
Epoch 4/100
13/13 [==============================] - 0s 21ms/step - loss: 1.3610 - val_loss: 1.2260
Epoch 5/100
13/13 [==============================] - 0s 13ms/step - loss: 1.1888 - val_loss: 1.1268
Epoch 6/100
13/13 [==============================] - 0s 21ms/step - loss: 1.0680 - val_loss: 1.1194
Epoch 7/100
13/13 [==============================] - 0s 20ms/step - loss: 0.9275 - val_loss: 0.8342
Epoch 8/100
13/13 [==============================] - 0s 13ms/step - loss: 0.7679 - val_loss: 0.7745
Epoch 9/100
13/13 [==============================] - 0s 13ms/step - loss: 0.6581 - val_loss: 0.6039
Epoch 10/100
13/13 [==============================] - 0s 14ms/step - loss: 0.5899 - val_loss: 0.5229
Epoch 11/10

In [ ]:
# Define the encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Decode sequence function
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<start>']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tokenizer.index_word.get(sampled_token_index, '')

        # Exit condition: either hit max length or find stop character.
        if decoded_sentence.endswith('<end>'):
            stop_condition = True
        else:
            decoded_sentence += sampled_char

        # Update the target sequence (length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.replace('<start>', '').replace('<end>', '').strip()


# Test decoding
for seq_index in range(10):
    input_seq = X[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('Input:', names[seq_index])
    print('Decoded:', decoded_sentence)


1/1 [==============================] - 0s 28ms/step
Input: dom.swaroop
Decoded: jack
1/1 [==============================] - 0s 28ms/step
Input: Mr. Alice
Decoded: alice
1/1 [==============================] - 0s 24ms/step
Input: Mr. Frank
Decoded: frank
1/1 [==============================] - 0s 25ms/step
Input: Grace
Decoded: grace
1/1 [==============================] - 0s 29ms/step
Input: Alice
Decoded: alice
1/1 [==============================] - 0s 30ms/step
Input: Mrs. Eve
Decoded: eve
1/1 [==============================] - 0s 29ms/step
Input: Frank
Decoded: frank
1/1 [==============================] - 0s 28ms/step
Input: Mrs. Alice
Decoded: alice
1/1 [==============================] - 0s 28ms/step
Input: Bob
Decoded: bob
1/1 [==============================] - 0s 28ms/step
Input: Alice
Decoded: alice
